In [2]:
import os
import pandas as pd

In [23]:
folder_c = "../src/model_ctypes/ina_pipette/"
legend_constants = pd.read_csv(os.path.join(folder_c, "legend_constants.csv"))
legend_states = pd.read_csv(os.path.join(folder_c, "legend_states.csv"))
legend_algebraic = pd.read_csv(os.path.join(folder_c, "legend_algebraic.csv"))

In [30]:
filename_model = os.path.join(folder_c, "ina.c")

lines_julia = []

with open(filename_model) as f:
    lines = f.readlines()
    
tuples = (legend_constants, "CONSTANTS"), (legend_states, "STATES"), (legend_algebraic, "ALGEBRAIC")
    
for line in lines:
    if line.strip().startswith("//"):
        continue
    for legend, name in tuples:
        for i_row, row in legend.iterrows():
            #print(i_row, row)
            old = f"{name}[{i_row}]"
            new = row["name"]
            if name == "CONSTANTS":
                new = "p." + new
            line = line.replace(old, new)
    line = line.strip()
    if line:
        lines_julia.append(line)

In [31]:
print("\n".join(lines_julia))

#include "ina.h"
#include "math.h"
void initialize_states_default(double *STATES) {
v_comp = -80;//v_comp
v_p = -80;//v_p
v_m = -80;//v_m
m = 0.;//m
h = 1.;//h
j = 1.;//j
I_out = 0;//I_out
}
void compute_algebraic(const double time,  double *STATES, double *CONSTANTS,  double *ALGEBRAIC){//, double *RATES){
tau_m = 1/(p.a0_m * exp(v_m/p.s_m) + p.b0_m*exp(- v_m/p.delta_m));
tau_h = 1/(p.a0_h * exp(- v_m/p.s_h) + p.b0_h*exp(v_m/p.delta_h));
tau_j = p.tau_j_const + 1/(p.a0_j * exp(- v_m/p.s_j) + p.b0_j*exp(v_m/p.delta_j));
m_inf = 1 / (1 + exp(-(p.v_half_m + v_m)/p.k_m));
h_inf = 1 / (1 + exp((p.v_half_h + v_m)/p.k_h));
v_cp = p.v_c + (p.v_c - v_comp)*(1/(1 - p.alpha) - 1);
I_leak = p.g_leak * v_m;
I_Na = p.g_max * h * pow(m,3) * j* (v_m - p.v_rev);
I_c = 1e9 * p.c_m * ((v_p + p.v_off - v_m)/(p.R*p.c_m) - (1e-9)*(I_leak + I_Na)/p.c_m);// RATES[2];
I_p = 1e9 * p.c_p *(v_cp - v_p)/(p.c_p*p.R_f);// RATES[1];
I_comp = 1e9 * p.x_c_comp * p.c_m *(p.v_c - v_comp)/(p.x_c_comp*p.c_m*p.x_r_comp*p.R

In [39]:
print(', '.join(legend_algebraic['name'].to_list()))

tau_m, tau_h, tau_j, m_inf, h_inf, v_cp, I_leak, I_Na, I_c, I_p, I_comp, I_in


In [46]:
for i_row, row in legend_constants.iterrows():
    print(f"{row['name']}={row.value}, ", end="")

c_p=4.502963e-12, c_m=2.473016e-11, a0_m=2736.956, b0_m=71.35904, delta_m=20.6157, s_m=9.306083, a0_h=0.4604052, b0_h=503.8082, delta_h=15.2348, s_h=21.96405, a0_j=0.4475461, b0_j=20613.33, delta_j=4.115303, s_j=167.3159, tau_j_const=0.0007794313, R=24946770.0, R_f=326421.4, g_max=439220.4, g_leak=0.4812443, tau_z=0.000100034, v_half_m=24.35802, v_half_h=59.69117, k_m=5.66677, k_h=5.507771, x_c_comp=0.09929933, x_r_comp=1.02, alpha=0.75, v_off=-2.0, v_rev=18.0, v_c=-80.0, 